Install the required packages:

In [1]:
!pip install -q -U bitsandbytes trl
!pip install -q datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 44.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyprojec

Import the required modules:

In [2]:
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from langdetect import detect, LangDetectException

Connect to Google Drive:

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/NLP/Project'

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/NLP/Project


In [4]:
# This function returns the language that is idenitfied for the given sentence
def detect_language(sentence):
    if isinstance(sentence, str):
        try:
            return detect(sentence)
        except LangDetectException:
            print(f"Unknown: {sentence}")
            return "unknown"
    else:
        print(f"Numbers: {sentence}")
        return "unknown"

In [5]:
def is_language(sentence, language):
    if detect_language(sentence) == language:
        return True
    else:
        return False

We load the openassistant-guanaco dataset:

In [6]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset_train = load_dataset(dataset_name, split="train") # We take only the train split of the dataset
dataset_test = load_dataset(dataset_name, split="test")

dataset_english_train = dataset_train.filter(lambda example: is_language(example['text'], "en"))
dataset_english_test = dataset_test.filter(lambda example: is_language(example['text'], "en"))

#dataset = dataset.select(range(10)) # We take only the first 10 rows

#train_dataset_csv_path = "Datasets/only_english_dataset_train.csv"
#test_dataset_csv_path = "Datasets/only_english_dataset_test.csv"
#dataset_train = load_dataset('csv', data_files = train_dataset_csv_path)
#dataset_test = load_dataset('csv', data_files = test_dataset_csv_path)

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.


Filter:   0%|          | 0/9846 [00:00<?, ? examples/s]

Unknown: ### Human: Generate me an ASCII of a person with a jetpack### Assistant: Here is an ASCII art representation of a person with a jetpack:

─────────▄███████████▄▄──────────────
──────▄██▀──────────▀▀██▄────────────
────▄█▀────────────────▀██───────────
──▄█▀────────────────────▀█▄─────────
─█▀──██──────────────██───▀██────────
█▀──────────────────────────██───────
█──███████████████████───────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█▄───────────────────────────█───────
▀█▄─────────────────────────██───────
─▀█▄───────────────────────██────────
──▀█▄────────────────────▄█▀─────────
───▀█▄──────────────────██───────────
─────▀█▄──────────────▄█▀────────────
───────▀█▄▄▄──────▄▄▄███████▄▄───────
────────███████████████───▀██████▄───
─────▄███▀▀────────▀███▄──────█─███──
───▄███▄─────▄▄▄▄────███────▄▄████▀──
─▄███▓▓█─────█▓▓█

Filter:   0%|          | 0/518 [00:00<?, ? examples/s]

Models to try: Falcon, LLM2, LLM3, gpt-2

We load he required model with 4-bit quantization:

In [7]:
#model_id = "EleutherAI/gpt-neox-20b"
model_id = "meta-llama/Meta-Llama-3-8B"

# We set the configuration for ourt quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We enable gradient checkpointing to save additional memory (it's a pretty standard thing to do) and we prepare the model for training (so we convert its parameters to 16 bit i think):

In [8]:
# Enabling the gradient checkpointing allow us to recompute some values of the neural network when needed, instead of storing them all and copnsuming a lot of memory
model.gradient_checkpointing_enable()

# This prepares the model for the kbit trainig
model = prepare_model_for_kbit_training(model)

Function to print the number and percentage of trainable parameters:

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

We set the config that are gonna be uses with LoRA:

In [10]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=[
    #   "query_key_value",
    #    "dense",
    #    "dense_h_to_4h",
    #    "dense_4h_to_h",
    #]
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

# And using this config we obtain the LoRA trainable version of the model
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 54525952 || all params: 4595126272 || trainable%: 1.1866039967660762


In [11]:
output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
#max_steps = 500
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    fp16 = True,
    max_grad_norm = max_grad_norm,
    max_steps = max_steps,
    warmup_ratio = warmup_ratio,
    group_by_length = True,
    lr_scheduler_type = lr_scheduler_type,
    gradient_checkpointing = True,
)

In [12]:
max_seq_length = 512

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset_english_train,
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = training_arguments,
)

Map:   0%|          | 0/3532 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [13]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.256800
20,1.268500
30,1.282900
40,1.522100
50,1.807100
60,1.216100
70,1.222600


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

Step,Training Loss
10,1.256800
20,1.268500
30,1.282900
40,1.522100
50,1.807100
60,1.216100
70,1.222600
80,1.267300
90,1.511400
100,1.636700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

TrainOutput(global_step=200, training_loss=1.3840096044540404, metrics={'train_runtime': 4519.4635, 'train_samples_per_second': 0.708, 'train_steps_per_second': 0.044, 'total_flos': 4.719546451378176e+16, 'train_loss': 1.3840096044540404, 'epoch': 0.9060022650056625})

In [15]:
# Save the model in a format suitable for inference
output_model_dir = "Saved_models/llama-8-finetuned-onlyEnglish"
model.save_pretrained(output_model_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Define your prompt
prompt = "### Human: Can you write a short introduction about the relevance of the term \"monopsony\" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research."

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

# Generate text based on the prompt
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1
)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print("Generated text:", generated_text)